# Preparing 8K rat genotypes with body lenght, BMI data

## Write samples file

First, we obtain a list of samples with body length and BMI data, then intersect with genotypes rats.

In [5]:
import pandas as pd
all_pheno = pd.read_csv("/home/s1mi/enformer_rat_data/phenotypes/ALLTRAITSALLNORMALIZES_19jul24.csv", 
                        usecols = ['rfid', 'dissection:regressedlr_length_w_tail_cm', 'dissection:regressedlr_bmi_w_tail'],
                        index_col = 'rfid')
all_pheno.columns = ['bodylen', 'bmi']
pheno_rats = all_pheno[all_pheno['bodylen'].notna()].index


/tmp/ipykernel_46928/766119701.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  all_pheno = pd.read_csv("/home/s1mi/enformer_rat_data/phenotypes/ALLTRAITSALLNORMALIZES_19jul24.csv",


In [ ]:
import pysam
vcf = pysam.VariantFile("/home/s1mi/enformer_rat_data/genotypes/ratgtex_v3_round10_5.rn7.vcf.gz")
geno_rats = list(vcf.header.samples)
samples = pheno_rats.intersection(geno_rats)
# with open("samples.txt", "w") as f:
    # f.write("\n".join(samples))



I downloaded the [bed, bim, fam files](https://library.ucsd.edu/dc/object/bb5610743d), then filtered to the 8K rats with phenotype data.

## Filter Samples

```
plink --bfile bb5610743d --keep-fam samples.txt --make-bed --out bodylen_bmi_8K_samples

```

## Convert NCBI RefSeq ID to chromosome numbers

```
bcftools annotate \
  --rename-chrs /home/s1mi/enformer_rat_data/annotation/mRatBN7.2_chromosome_names.txt \
  --threads 20 \
  -o /eagle/AIHPC4Edu/sabrina/scratch/ratgtex_v3_round10_5_annot.rn7.vcf \
  -Ov /home/s1mi/enformer_rat_data/genotypes/ratgtex_v3_round10_5.rn7.vcf.gz

```

## Replace Missing Genotypes with Reference Alleles

```
sed 's/\.\/\./0\/0/g' ratgtex_v3_round10_5_annot.rn7.vcf > ratgtex_v3_round10_5_no_missing.vcf
```

### Compute Haplotype Probabilities

```
conda activate genomics
cd Github/deep-learning-in-genomics/posts/2023-11-22-qtl2-founder-haps-pbs-job-array
GENO_DIR=/Users/sabrinami/Desktop/Sabrina/2022-23/tutorials/enformer_pipeline_test/rn7_data
DATA_DIR=/Users/sabrinami/Desktop/qtl2_data
for CHR in {1..20}
do
    python make_qtl2_inputs.py $GENO_DIR/BrainVCFs/chr${CHR}.vcf.gz $GENO_DIR/FounderVCFs/chr${CHR}.vcf.gz --working-dir $DATA_DIR/chr${CHR}-qtl2-founder-haps --gmap-dir $DATA_DIR/genetic_map
done

```